# Update

In [1]:
import pandas as pd

## Database connection

In [2]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [3]:
update_dimension = pd.read_sql_table("mensajeria_tiponovedad", OLTP_connection)

## Transformation

In [5]:
update_dimension.drop(columns=["id"], inplace=True)
update_dimension.reset_index(inplace=True)
update_dimension.rename(
	columns={
		"index": "update_id",
		"nombre": "update_name",
	}, inplace=True
)
update_dimension.set_index("update_id", inplace=True)

## Load

In [7]:
update_dimension.to_sql(
	"UPDATE_DIMENSION", OLAP_connection, if_exists="replace", index=True
)

2